In [25]:
import os
from PIL import Image
from skimage import io
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2

In [ ]:
# Data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)



model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1), kernel_regularizer=L2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=L2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=L2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=L2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(4, activation='softmax')
])


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

Found 4672 images belonging to 4 classes.
Found 576 images belonging to 4 classes.
Found 576 images belonging to 4 classes.


C:\Users\Arjun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Define directories
base_dir = 'brain_tumor_mris'
sub_dirs = ['Testing', 'Training', 'Validation']
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Initialize variables to store minimum width and height
min_width = float('inf')
min_height = float('inf')

# Iterate through all images to find the minimum width and height
for sub_dir in sub_dirs:
    for category in categories:
        path = os.path.join(base_dir, sub_dir, category)
        for img in os.listdir(path):
            try:
                img_path = os.path.join(path, img)
                image = io.imread(img_path, as_gray=True)
                height, width = image.shape
                if width < min_width:
                    min_width = width
                if height < min_height:
                    min_height = height
            except Exception as e:
                print(f"Error loading image {img}: {e}")

print(f"Minimum width: {min_width}")
print(f"Minimum height: {min_height}")

#min width 150
#max height 168

In [ ]:
# Define directories
base_dir = 'brain_tumor_mris'
output_dir = 'resized_brain_tumor_mris'
sub_dirs = ['Testing', 'Training', 'Validation']
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Create output directories if they don't exist
for sub_dir in sub_dirs:
    for category in categories:
        os.makedirs(os.path.join(output_dir, sub_dir, category), exist_ok=True)

# Function to resize images
def resize_image(image_path, size=(150, 150)):
    with Image.open(image_path) as img:
        # Convert image to RGB if it's in a different mode
        if img.mode != 'RGB':
            img = img.convert('RGB')
        resized_img = img.resize(size)
        return resized_img

# Iterate through all images and resize them
for sub_dir in sub_dirs:
    for category in categories:
        input_path = os.path.join(base_dir, sub_dir, category)
        output_path = os.path.join(output_dir, sub_dir, category)
        for img in os.listdir(input_path):
            try:
                img_path = os.path.join(input_path, img)
                resized_img = resize_image(img_path)
                resized_img.save(os.path.join(output_path, img))  # Save resized image as a copy
            except Exception as e:
                print(f"Error processing image {img}: {e}")

print("All images have been resized to 150x150 pixels and saved as copies.")


In [41]:
# Define directories
train_dir = 'resized_brain_tumor_mris/Training'
val_dir = 'resized_brain_tumor_mris/Validation'
test_dir = 'resized_brain_tumor_mris/Testing'


In [43]:
# Data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)



model = Sequential([
    Conv2D(32, (2, 2), activation='relu', input_shape=(150, 150, 1), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (2, 2), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (2, 2), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(4, activation='softmax')
])



# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['f1_score'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

Found 4672 images belonging to 4 classes.
Found 576 images belonging to 4 classes.
Found 576 images belonging to 4 classes.


In [44]:
# Train the model with early stopping
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

# Save the model
model.save('brain_tumor_classifier.keras')


Epoch 1/20
146/146 ━━━━━━━━━━━━━━━━━━━━ 42s 274ms/step - f1_score: 0.5247 - loss: 1.8260 - val_f1_score: 0.1135 - val_loss: 14.9467
Epoch 2/20
146/146 ━━━━━━━━━━━━━━━━━━━━ 45s 308ms/step - f1_score: 0.6632 - loss: 1.4079 - val_f1_score: 0.1000 - val_loss: 37.0714
Epoch 3/20
146/146 ━━━━━━━━━━━━━━━━━━━━ 43s 289ms/step - f1_score: 0.6876 - loss: 1.3002 - val_f1_score: 0.1184 - val_loss: 7.8516
Epoch 4/20
146/146 ━━━━━━━━━━━━━━━━━━━━ 43s 291ms/step - f1_score: 0.6922 - loss: 1.2738 - val_f1_score: 0.4005 - val_loss: 1.9589
Epoch 5/20
146/146 ━━━━━━━━━━━━━━━━━━━━ 44s 300ms/step - f1_score: 0.7066 - loss: 1.1954 - val_f1_score: 0.4855 - val_loss: 1.5811
Epoch 6/20
146/146 ━━━━━━━━━━━━━━━━━━━━ 43s 291ms/step - f1_score: 0.7342 - loss: 1.1268 - val_f1_score: 0.4975 - val_loss: 1.9238
Epoch 7/20
146/146 ━━━━━━━━━━━━━━━━━━━━ 42s 287ms/step - f1_score: 0.7535 - loss: 1.0628 - val_f1_score: 0.3198 - val_loss: 2.9428
Epoch 8/20
146/146 ━━━━━━━━━━━━━━━━━━━━ 42s 285ms/step - f1_score: 0.7661 - loss: